In [ ]:
import requests
import pandas as pd
import json
import plotly.express as px
from plotly.subplots import make_subplots

# Obtenemos las claves de acceso a la API
url = "https://api.invertironline.com/token"

payload = 'username={tu_usuario}&password={tu_contraseña}*&grant_type=password'
headers = {
  'Content-Type': 'application/x-www-form-urlencoded',
  'Cookie': '1ea603=90N+IRNhzS+ZAlji2JeWwwFHYqnB5ZldfmpPr25mmXPfJx3JyDKWDLm2ANNR0cnLgvJJGe0qOY6g/L/4ppP/uvsPRie6OCjdqveFKwQxSr1ezBsEsymWYUkPcg9hDa8WUnpyEbGvdOsRU3DCvgzlkClGmeRHinBuMlRNLwRye7fv0V3+'
}

response = requests.request("POST", url, headers=headers, data=payload)
# Convertimos la respuesta a JSON
response_json = response.json()

# KEYS
bearer_token = response_json["access_token"]
refresh_token = response_json["refresh_token"]

try:
    url = "https://api.invertironline.com/api/portafolio"

    payload = {}
    headers = {
    'Authorization': f"Bearer {bearer_token}",
    'Cookie': '1ea603=5qKTn+1drTePYyM4PtURnV5nPh4UJIvPk+LaeimycJoK3WjrL4qjWlt9oRKXSMDgE9wnQ1eidll6g3hpm+TUvkl6UXgeRnGxxb7KDQTAFTZp3Xx0KPEtP7CC8d7rKmtYrPO2rNJywJ3qgQbnBpcG4o463FxwUWOptCxodnPU9XkAfRTT'
    }

    response_json = requests.request("GET", url, headers=headers, data=payload).json()

    portafolio_json = json.dumps(response_json, indent=2)

except Exception as e:
    print(f"Error al obtener el portafolio: {e}")

ticket = []
tipo = []
cantidad = []
ultimo_precio = []
ganancia_porcentual = []
ganancia_monetaria = []
valor_total = []

data = json.loads(portafolio_json)

activos = data["activos"]

for activo in activos:
    
    ticket1 = activo["titulo"]["simbolo"]
    ticket.append(ticket1)
    
    tipo1 = activo["titulo"]["tipo"]
    tipo.append(tipo1)
    
    cantidad1 = activo["cantidad"]
    cantidad.append(cantidad1)
    
    ultimo_precio1 = activo["ultimoPrecio"]
    ultimo_precio.append(ultimo_precio1)
    
    ganancia_porcentual1 = activo["gananciaPorcentaje"]
    ganancia_porcentual.append(ganancia_porcentual1)
    
    ganancia_monetaria1 = activo["gananciaDinero"]
    ganancia_monetaria.append(ganancia_monetaria1)
    
    valor_total1 = activo["valorizado"]
    valor_total.append(valor_total1)
    
portafolio_df = pd.DataFrame({
    "ticket" : ticket, 
    "tipo" : tipo,
    "cantidad" : cantidad, 
    "ultimo_precio" : ultimo_precio, 
    "ganancia_porcentual" : ganancia_porcentual, 
    "ganancia_monetaria" : ganancia_monetaria, 
    "valor_total" : valor_total 
})

compo_portafolio = px.pie(portafolio_df, 
                          names='tipo', 
                          values='valor_total')

ganancia_por_tipo = px.bar(portafolio_df,
                           x="tipo",  # Agrupación por tipo de activo
                           y="ganancia_porcentual",
                           color="ticket",  # Diferenciar cada activo
                           barmode="group",  # Barras agrupadas
                           color_discrete_sequence=px.colors.qualitative.Set1,
                           text="ticket")
ganancia_por_tipo.update_traces(textfont_size=12, textangle=0, textposition="inside", cliponaxis=False)
ganancia_por_tipo.update_layout(bargap=0.2)

ganancia_por_ticket = px.bar(portafolio_df,
                       x="ticket",
                       y="ganancia_monetaria",
                       color="ticket")
ganancia_por_ticket.update_layout(bargap=0.1) 

# Crear el subplot
fig_informe = make_subplots(
    rows=2, cols=2,
    specs=[[{"type": "pie"}, None], [{"type": "bar"}, {"type": "bar"}]],
    subplot_titles=("Composición del portafolio", "Ganancia % por tipo de activo", "Ganancia monetaria por ticket"),
    horizontal_spacing=0.1,
    vertical_spacing=0.1
)

# Ajustar la distancia entre los títulos de los subplots y los gráficos
for annotation in fig_informe['layout']['annotations']:
    annotation['y'] += 0.02  # Ajusta hacia arriba (incrementa) o hacia abajo (disminuye)
    annotation['font'] = dict(size=15, color="white", family="Arial")  # Personalización opcional


# Añadir la traza del pie chart al subplot (mostrar leyenda solo aquí)
compo_portafolio.data[0]['showlegend'] = True  # Mostrar leyenda del pie chart
fig_informe.add_trace(compo_portafolio.data[0], row=1, col=1)

# Añadir trazas del barplot Ganancia % por tipo (ocultar leyenda)
for tipo in ganancia_por_tipo.data:
    tipo['showlegend'] = False  # Ocultar leyenda de cada traza
    fig_informe.add_trace(tipo, row=2, col=1)

# Añadir trazas del barplot Ganancia monetaria por ticket (ocultar leyenda)
for tick in ganancia_por_ticket.data:
    tick['showlegend'] = False  # Ocultar leyenda de cada traza
    fig_informe.add_trace(tick, row=2, col=2)

# Configuración general del layout
fig_informe.update_layout(
    title_text="Mi portafolio",
    height=800,
    template="plotly_dark",
    legend=dict(
        title="Tipos de Activo",  # Título de la leyenda
        x=0.6,  # Posición horizontal
        y=0.9,  # Posición vertical
        font=dict(
            size=12,
            color="white",
            family="Arial"),
        bgcolor="rgba(0, 0, 0, 0.5)",
        bordercolor="white",
        borderwidth=0.5,
        orientation="v"
    ),
    bargap=0.0,
    bargroupgap=0.1
)

# Mostrar el gráfico
fig_informe.show()